In [56]:
import numpy as np

# CONDENSE TREE - INCOMPLETE

In [57]:
hierarchy = np.array([
        [ 3,          7,          0.4667349,   2        ],
        [10,          5,          0.51406193,  3        ],
        [ 1,          6,          0.53936375,  2        ],
        [ 2,         11,          0.61524793,  4        ],
        [13,          4,          0.61524793,  5        ],
        [12,          8,          0.69040949,  3        ],
        [ 0,         15,          0.76419459,  4        ],
        [16,          9,          0.76419459,  5        ],
        [17,         14,          2.02808855,  10       ]
    ])

min_cluster_size=10
hierarchy

array([[ 3.        ,  7.        ,  0.4667349 ,  2.        ],
       [10.        ,  5.        ,  0.51406193,  3.        ],
       [ 1.        ,  6.        ,  0.53936375,  2.        ],
       [ 2.        , 11.        ,  0.61524793,  4.        ],
       [13.        ,  4.        ,  0.61524793,  5.        ],
       [12.        ,  8.        ,  0.69040949,  3.        ],
       [ 0.        , 15.        ,  0.76419459,  4.        ],
       [16.        ,  9.        ,  0.76419459,  5.        ],
       [17.        , 14.        ,  2.02808855, 10.        ]])

In [58]:
def bfs_from_hierarchy(hierarchy, root):

    to_process = []
    dim = hierarchy.shape[0]
    max_node = 2 * dim
    num_points = max_node - dim + 1
    
    to_process = [int(root)]
    result = []


    while to_process:
        result.extend(to_process)
        to_process = [x - num_points for x in
                        to_process if x >= num_points]
        if to_process:
            
            to_process = hierarchy[to_process,
                                    :2].flatten().astype(np.intp).tolist()
    return result

In [59]:
INFTY = np.inf
root = 2 * hierarchy.shape[0]
num_points = root // 2 + 1
next_label = num_points + 1

node_list = bfs_from_hierarchy(hierarchy, root)
print(node_list)
print(root)

[18, 17, 14, 16, 9, 13, 4, 0, 15, 2, 11, 12, 8, 10, 5, 1, 6, 3, 7]
18


In [60]:
relabel = np.empty(root + 1)
relabel[root] = num_points
result_list = []
ignore = np.zeros(len(node_list))

for node in node_list:
    if ignore[node] or node < num_points:
        continue

    children = hierarchy[node - num_points]
    left = children[0]
    right = children[1]
    if children[2] > 0.0:
        lambda_value = 1.0 / children[2]
    else:
        lambda_value = INFTY

    if left >= num_points:
        left_count = hierarchy[int(left - num_points)][3]
    else:
        left_count = 1

    if right >= num_points:
        right_count = hierarchy[int(right - num_points)][3]
    else:
        right_count = 1

    if left_count >= min_cluster_size and right_count >= min_cluster_size:
        relabel[left] = next_label
        next_label += 1
        result_list.append((relabel[node], relabel[left], lambda_value,
                            left_count))

        relabel[right] = next_label
        next_label += 1
        result_list.append((relabel[node], relabel[right], lambda_value,
                            right_count))

    elif left_count < min_cluster_size and right_count < min_cluster_size:
        for sub_node in bfs_from_hierarchy(hierarchy, left):
            if sub_node < num_points:
                result_list.append((relabel[node], sub_node,
                                    lambda_value, 1))
            ignore[sub_node] = True

        for sub_node in bfs_from_hierarchy(hierarchy, right):
            if sub_node < num_points:
                result_list.append((relabel[node], sub_node,
                                    lambda_value, 1))
            ignore[sub_node] = True

    elif left_count < min_cluster_size:
        relabel[right] = relabel[node]
        for sub_node in bfs_from_hierarchy(hierarchy, left):
            if sub_node < num_points:
                result_list.append((relabel[node], sub_node,
                                    lambda_value, 1))
            ignore[sub_node] = True

    else:
        relabel[left] = relabel[node]
        for sub_node in bfs_from_hierarchy(hierarchy, right):
            if sub_node < num_points:
                result_list.append((relabel[node], sub_node,
                                    lambda_value, 1))
            ignore[sub_node] = True


    # return np.array(result_list, dtype=[('parent', np.intp),
                                        # ('child', np.intp),
                                        # ('lambda_val', float),
                                        # ('child_size', np.intp)])


    # print(result_list)

In [61]:
condensed_tree = np.array(result_list)
condensed_tree

array([[10.        ,  9.        ,  0.49307512,  1.        ],
       [10.        ,  0.        ,  0.49307512,  1.        ],
       [10.        ,  8.        ,  0.49307512,  1.        ],
       [10.        ,  1.        ,  0.49307512,  1.        ],
       [10.        ,  6.        ,  0.49307512,  1.        ],
       [10.        ,  4.        ,  0.49307512,  1.        ],
       [10.        ,  2.        ,  0.49307512,  1.        ],
       [10.        ,  5.        ,  0.49307512,  1.        ],
       [10.        ,  3.        ,  0.49307512,  1.        ],
       [10.        ,  7.        ,  0.49307512,  1.        ]])

# COMPUTE STABILITY - INCOMPLETE

In [62]:
condensed_tree = np.array([
    (10, 11, 0.49307512, 5), (10, 12, 0.49307512, 5), (11,  0, 1.30856724, 1),
    (11,  8, 1.30856724, 1), (11,  1, 1.30856724, 1), (11,  6, 1.30856724, 1),
    (11,  9, 1.30856724, 1), (12,  2, 1.62536102, 1), (12,  5, 1.62536102, 1),
    (12,  3, 1.62536102, 1), (12,  7, 1.62536102, 1), (12,  4, 1.62536102, 1)])
condensed_tree

array([[10.        , 11.        ,  0.49307512,  5.        ],
       [10.        , 12.        ,  0.49307512,  5.        ],
       [11.        ,  0.        ,  1.30856724,  1.        ],
       [11.        ,  8.        ,  1.30856724,  1.        ],
       [11.        ,  1.        ,  1.30856724,  1.        ],
       [11.        ,  6.        ,  1.30856724,  1.        ],
       [11.        ,  9.        ,  1.30856724,  1.        ],
       [12.        ,  2.        ,  1.62536102,  1.        ],
       [12.        ,  5.        ,  1.62536102,  1.        ],
       [12.        ,  3.        ,  1.62536102,  1.        ],
       [12.        ,  7.        ,  1.62536102,  1.        ],
       [12.        ,  4.        ,  1.62536102,  1.        ]])

In [63]:
largest_child = condensed_tree[:,1].max()
# largest_child = condensed_tree['child'].max()
smallest_cluster = condensed_tree[:,0].min()
# smallest_cluster = condensed_tree['parent'].min()

num_clusters = (condensed_tree[:,0].max() - smallest_cluster + 1)


In [64]:
if largest_child < smallest_cluster:
    largest_child = smallest_cluster

In [65]:
sorted_child_data = np.sort(condensed_tree[:,[1,2]],
                            axis=0)
sorted_children = sorted_child_data[:,0].copy()
sorted_lambdas = sorted_child_data[:,1].copy()
births_arr = np.nan * np.ones(int(largest_child) + 1)
births = births_arr.copy()
# births_arr = np.nan * np.ones(int(largest_child) + 1)
# births = (<np.double_t *> births_arr.data)
# sorted_children = sorted_child_data['child'].copy()
# sorted_lambdas = sorted_child_data['lambda_val'].copy()

In [66]:
parents = condensed_tree[:,0]
sizes = condensed_tree[:,3]
lambdas = condensed_tree[:,2]

# parents = condensed_tree['parent']
# sizes = condensed_tree['child_size']
# lambdas = condensed_tree['lambda_val']

In [67]:
current_child = -1
min_lambda = 0

In [68]:
for row in range(sorted_child_data.shape[0]):
    child = sorted_children[row]
    lambda_ = sorted_lambdas[row]

    if child == current_child:
        min_lambda = min(min_lambda, lambda_)
    elif current_child != -1:
        births[int(current_child)] = min_lambda
        current_child = child
        min_lambda = lambda_
    else:
        # Initialize
        current_child = child
        min_lambda = lambda_
if current_child != -1:
    births[int(current_child)] = min_lambda
births[int(smallest_cluster)] = 0.0

In [69]:
#logic for choosing stable clusters
result_arr = np.zeros(int(num_clusters))


for i in range(condensed_tree.shape[0]):
    parent = parents[i]
    lambda_ = lambdas[i]
    child_size = sizes[i]
    result_index = parent - smallest_cluster

    result_arr[int(result_index)] += (lambda_ - births[int(parent)]) * child_size


In [70]:
result_pre_dict = np.vstack((np.arange((smallest_cluster),
                                        condensed_tree[:,0].max() + 1),
                                result_arr)).T


In [71]:
result_pre_dict

array([[10.       ,  4.9307512],
       [11.       , -1.5839689],
       [12.       ,  0.       ]])

In [72]:
result_arr

array([ 4.9307512, -1.5839689,  0.       ])